In [39]:
#Import the PuLP library.
from pulp import *


In [40]:
# Define a dictionary named workers that contains the names of different types of workers and their hourly rates.
workers = {
    'ProjectManager': 72,  # Example hourly rate for the project manager
    'FrontendDeveloper': 60,
    'BackendDeveloper': 60,
    'DataScientist': 58,
    'DataEngineer': 72,
}

In [41]:
# Define a dictionary named activities that contains the names of project tasks and their estimated durations (in hours) for the "BestCaseTime" scenario.
activities = {
    'Describe product': {'BestCaseTime': 2},
    'Develop marketing strategy': {'BestCaseTime': 8},
    'Design brochure': {'BestCaseTime': 6},
    #'Develop product protoype': {'BestCaseTime': 40},
    'Requirements analysis': {'BestCaseTime': 6},
    'Software design': {'BestCaseTime': 8},
    'System design':{'BestCaseTime': 8}, 
    'Coding': {'BestCaseTime': 20},
    'Write documentation': {'BestCaseTime': 10},
    'Unit testing': {'BestCaseTime': 10},
    'System testing': {'BestCaseTime': 12},
    'Package deliverables': {'BestCaseTime': 4},
    'Survey potential market': {'BestCaseTime': 10},
    'Develop pricing plan': {'BestCaseTime': 6},
    'Develop implementation  plan': {'BestCaseTime': 8},
    'Write client proposal': {'BestCaseTime': 4}

}


In [42]:
# Create a list named activities_list containing the names of all project tasks.
activities_list = list(activities.keys())


In [43]:
#Define a dictionary named precedences that specifies the precedence relationships between tasks. Tasks are organized hierarchically, with some tasks depending on others.
precedences = {
            'Describe product': [],
            'Develop marketing strategy': [], 
            'Design brochure': ['Describe product'], 
            #'Develop product protoype': [],  
            'Requirements analysis': ['Describe product'], 
            'Software design': ['Requirements analysis'], 
            'System design': ['Requirements analysis'], 
            'Coding': ['Software design','System design'],
            'Write documentation': ['Coding'],
            'Unit testing': ['Coding'],
            'System testing' : ['Unit testing'],
            'Package deliverables': ['Write documentation','System testing'],
            'Survey potential market':['Develop marketing strategy','Design brochure'],
            'Develop pricing plan': ['Package deliverables','Survey potential market'],
            'Develop implementation  plan': ['Describe product','Package deliverables'],
            'Write client proposal': ['Develop pricing plan','Develop implementation  plan']
            }


In [44]:
# Create the LP problem named "Critical Path" 
prob = LpProblem("Critical Path", LpMinimize)


In [45]:
# Create the LP variables
#Define LP variables start_times and end_times to represent the start and end times of each task.
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}


In [46]:
# Add constraints to ensure that the end time of each task is equal to the start time plus the task's duration.
# Add constraints to enforce precedence relationships between tasks. These constraints ensure that a task cannot start until its predecessors have finished.
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"


In [47]:
#Define the objective function to minimize the total project duration (in days) by summing the end times of all tasks.
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} hours in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/a931019fc99f41d18adabb3d8564392b-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/a931019fc99f41d18adabb3d8564392b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 39 COLUMNS
At line 123 RHS
At line 158 BOUNDS
At line 159 ENDATA
Problem MODEL has 34 rows, 30 columns and 68 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-34) rows, 0 (-30) columns and 0 (-68) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 536
After Postsolve, objective 536, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 536 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions changed 